In [28]:
# i know i repeated myself
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

In [2]:
df = pd.read_csv('../DSI-CHI-1/lessons/week-06/3.1-random-forests-boosting/assets/datasets/car.csv')

In [3]:
df.head(2)

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc


Prepare the data

Load the data into a pandas dataframe

Encode the categorical features properly: 
define a map that preserves the scale (assigning smaller numbers to words indicating smaller quantities)

Separate features from target into X and y

In [5]:
y = LabelEncoder().fit_transform(df['acceptability'])
X = pd.get_dummies(df.drop('acceptability', axis=1))

In [7]:
y

array([2, 2, 2, ..., 2, 1, 3])

In [9]:
# dropped target from df and made dummie_var for our columns and expanded options in those columns to be variables
X.head(2)

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


Useful preparation

Separate X and y between a train and test set, using 30% test set, random state = 42
make sure that the data is shuffled and stratified

Define a function called evaluate_model that trains the model on the train set tests it on the test and then calculates:

accuracy score

confusion matrix

classification report

Initialize a global dictionary to store the various models for later retrieval

In [11]:
# setting up train and test data for X and y
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(1209, 7) (1209,)
(519, 7) (519,)


In [13]:
# data is shuffled
cv = KFold(len(df),n_folds=3, shuffle=True, random_state=42)

In [14]:
cv

sklearn.cross_validation.KFold(n=1728, n_folds=3, shuffle=True, random_state=42)

In [15]:
KFold?

In [17]:
StratifiedKFold?

In [26]:
# stratified and shuffled these are the ones for train and test set
cv = StratifiedKFold(y, n_folds=3, shuffle=True, random_state=42)

In [27]:
cv

sklearn.cross_validation.StratifiedKFold(labels=[2 2 2 ..., 2 1 3], n_folds=3, shuffle=True, random_state=42)

In [29]:
def evaluate_model(X_train, X_test, 
                   y_train, y_test,
                  model):
    model.fit(x_train, y_train)
    print model.score(x_test, y_test)
    predictions = model.predict(X_test)
    # find confuse, class

In [30]:
# cross val list of scores is accuracy of each folds. avg cv score of all folds
# def score(model, name):
#     s = cross_val_score(model, X, y, cv=cv, n_jobs=-1)
#     print "{} Score:\t{:0.3} ± {:0.3}".format(name, s.mean().round(3), s.std().round(3))

In [32]:
dt = DecisionTreeClassifier(class_weight='balanced')
# takes model i set up and takes x and y's and runs each fold through model
# we want to see more though
s = cross_val_score(dt, X, y, cv=cv, n_jobs=-1)

In [33]:
print "{} Score:\t{:0.3} ± {:0.3}".format("Decision Tree", s.mean().round(3), s.std().round(3))

Decision Tree Score:	0.954 ± 0.008


In [34]:
bdt = BaggingClassifier(DecisionTreeClassifier())

In [35]:
rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

In [36]:
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)

In [37]:
score(dt, "Decision Tree")

Decision Tree Score:	0.954 ± 0.008


In [38]:
score(bdt, "Bagging DT")

Bagging DT Score:	0.969 ± 0.007


In [39]:
score(rf, "Random Forest")

Random Forest Score:	0.94 ± 0.004


In [40]:
score(et, "Extra Trees")

Extra Trees Score:	0.953 ± 0.01


In [41]:
# want to build a global dict to store all my models for future use
global_models = {'bdt': BaggingClassifier(DecisionTreeClassifier()), 
                'rf': RandomForestClassifier(class_weight='balanced', n_jobs=-1),
                'et': ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)}    

In [ ]:
def confuse_model: